# Time Series predictions using LSTM

Yet another try

In [1]:
import pandas as pd
from pandas_datareader import data, wb
from datetime import datetime
import numpy as np
import graphviz
import random

import sys
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import bokeh.models
import bokeh.plotting as bk
bk.output_notebook()

import ts_helpers

Loading BokehJS ...

In [2]:
np.random.seed(42)
random.seed(42)

WINDOW_IN = 7
WINDOW_OUT = 4

TICKERS = [pd.read_csv('tickers/%s' % f)
           for f in os.listdir('tickers')
           if f.endswith('.csv')
          ]
TICKERS = [ts_helpers.build_corpus(
                    df,
                    wnd_in=WINDOW_IN,
                    wnd_out=WINDOW_OUT)
            for df in TICKERS
            if df.shape[0] >= 364
]

In [3]:
class RNN(nn.Module): 
    
    def __init__(self, n_in, n_mem, days_out):
        super(RNN, self).__init__()
        self.n_in, self.n_mem, self.days_out = n_in, n_mem, days_out
        self.xm2f = nn.Linear(in_features=(n_mem + n_in), out_features=n_mem)
        self.xm2invent = nn.Linear(in_features=(n_mem + n_in), out_features=n_mem)
        self.xm2save = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.x2mid = nn.Linear(in_features=n_in, out_features=n_mem)
        self.mid2o = nn.Linear(in_features=n_mem, out_features=days_out)
        
    def init_weights(self, var=.1):
        for tr in [
                self.xm2f,
                self.xm2invent,
                self.xm2save,
                self.x2mid,
                self.mid2o,
                ]:
            tr.weight.data.uniform_(-var, var)
    
    def forward(self, x, mem):
        xmem = torch.cat((x, mem))
        mem = mem * F.sigmoid(self.xm2f(xmem))
        new_mem = F.tanh(self.xm2invent(xmem))
        mem_mask = F.sigmoid(self.xm2save(xmem))
        mem = ((1 - mem_mask) * mem
               + mem_mask * new_mem
        )
        mid = self.x2mid(x) * mem
        o = self.mid2o(mid)
        sgn = torch.stack((F.sigmoid(-o), F.sigmoid(o)), dim=1)
        return (o, sgn, mem)
    
    def init_mem(self):
        return Variable(torch.zeros(self.n_mem))

In [4]:
OBJECTIVE = ts_helpers.my_objective

In [5]:
rnn = RNN(n_in=TICKERS[0][0].shape[1], n_mem=64, days_out=TICKERS[0][1].shape[1])
rnn.init_weights(.1)
optimizer = torch.optim.RMSprop(rnn.parameters(), lr=.001)

In [6]:
from sklearn.model_selection import train_test_split
TR, TE = train_test_split(TICKERS)

In [7]:
def get_tr():
    return random.choice(TR)

def get_te():
    return random.choice(TE)

In [8]:
def train(model, epochs=2):
    for epoch in range(1, epochs+1):
        if epoch % 4 == 1:
            X, y = get_tr()
        print(
            'Epoch #%s; Loss: %s'
            % (
                epoch,
                ts_helpers.loss(model, X, y,
                     OBJECTIVE,
                     optimizer)))

In [9]:
train(rnn, epochs=50)

Epoch #1; Loss: 1.0615330934524536
Epoch #2; Loss: 1.0556527376174927
Epoch #3; Loss: 1.0509520769119263
Epoch #4; Loss: 1.0462576150894165
Epoch #5; Loss: 0.899976372718811
Epoch #6; Loss: 0.8983178734779358
Epoch #7; Loss: 0.8980128765106201
Epoch #8; Loss: 0.8978170156478882
Epoch #9; Loss: 1.3240978717803955
Epoch #10; Loss: 1.3113149404525757
Epoch #11; Loss: 1.308531403541565
Epoch #12; Loss: 1.3061161041259766
Epoch #13; Loss: 1.8838719129562378
Epoch #14; Loss: 1.8633171319961548
Epoch #15; Loss: 1.8563073873519897
Epoch #16; Loss: 1.8494523763656616
Epoch #17; Loss: 1.0326229333877563
Epoch #18; Loss: 1.0303738117218018
Epoch #19; Loss: 1.029458999633789
Epoch #20; Loss: 1.0284804105758667
Epoch #21; Loss: 1.0355602502822876
Epoch #22; Loss: 1.0339531898498535
Epoch #23; Loss: 1.0329713821411133
Epoch #24; Loss: 1.032209873199463
Epoch #25; Loss: 1.4881383180618286
Epoch #26; Loss: 1.4796149730682373
Epoch #27; Loss: 1.4745885133743286
Epoch #28; Loss: 1.4693471193313599
Epoch

In [10]:
Xte, yte = get_te()
ts_helpers.evaluate_model(rnn, Xte, yte,
               include_plohi=False,
               title='Evaluation on random test set')

In [ ]:
Xte, yte = get_te()
ts_helpers.evaluate_model(rnn, Xte, yte,
               include_plohi=True, include_tlohi=True,
               title='Evaluation on the test set')

In [ ]:
train(rnn, epochs=100)

Epoch #1; Loss: 0.8861274719238281
Epoch #2; Loss: 0.8860247731208801
Epoch #3; Loss: 0.885956346988678
Epoch #4; Loss: 0.8858857154846191
Epoch #5; Loss: 1.3088748455047607
Epoch #6; Loss: 1.302379846572876
Epoch #7; Loss: 1.299346685409546
Epoch #8; Loss: 1.2965079545974731
Epoch #9; Loss: 1.0282965898513794
Epoch #10; Loss: 1.018760323524475
Epoch #11; Loss: 1.0136557817459106
Epoch #12; Loss: 1.0092391967773438
Epoch #13; Loss: 4.534051895141602
Epoch #14; Loss: 4.35496187210083
Epoch #15; Loss: 4.274072647094727
Epoch #16; Loss: 4.192424774169922
Epoch #17; Loss: 6.198784828186035
Epoch #18; Loss: 5.504373073577881
Epoch #19; Loss: 5.157756805419922
Epoch #20; Loss: 4.8679938316345215
Epoch #21; Loss: 9.031198501586914
Epoch #22; Loss: 8.051976203918457
Epoch #23; Loss: 7.61358118057251
Epoch #24; Loss: 7.292810916900635
Epoch #25; Loss: 0.9257843494415283
Epoch #26; Loss: 0.9244466423988342
Epoch #27; Loss: 0.9237539172172546
Epoch #28; Loss: 0.9233269095420837
Epoch #29; Loss: 3

In [ ]:
Xte, yte = get_te()
ts_helpers.evaluate_model(rnn, Xte, yte,
               include_tlohi=True,
               title='Evaluation on a random test set')

In [ ]:
train(rnn, epochs=200)

In [ ]:
Xte, yte = get_te()
ts_helpers.evaluate_model(rnn, Xte, yte,
               include_tlohi=True,
               title='Evaluation on a test set')

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

In [ ]:
torch.save(rnn.state_dict(), nb_name + '.weights')

In [ ]:
train(rnn, epochs=500)

In [ ]:
Xte, yte = get_te()
ts_helpers.evaluate_model(rnn, Xte, yte,
               include_tlohi=True,
               title='Evaluation on a random test set')